In [1]:
import numpy as np
# from lab1.mobile_kinematic import Diff_Drive_Kinematic

In [9]:
numl = [0, 1, 2, 3, 4, 5]
num = np.arange(0, 16)
for i in num:
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [3]:
class Diff_Drive_Kinematic():
    def __init__(self, r : float, b : float):
        self.Tt2w = np.array([1.0 / r, -0.5 * b / r,
                         1.0 / r, 0.5 * b / r]).reshape(2, 2)
        self.Tw2t = np.array([0.5 * r, 0.5 * r,
                              0.0, 0.0, 
                              -r / b, r / b]).reshape(3, 2)
        self.pos_g = np.zeros(3)
        self.vel_g = np.zeros(3)
        self.qd = np.zeros(2)

    def get_wheelspeed(self, vel_r : list):
        self.qd = self.Tt2w @ np.array(vel_r)
        return self.qd

    def get_pose(self, dt : float):
        Rr2g = np.array([np.cos(self.pos_g[2]), -np.sin(self.pos_g[2]), 0.0,
                         np.sin(self.pos_g[2]), np.cos(self.pos_g[2]), 0.0,
                         0.0, 0.0, 1.0]).reshape(3, 3)
        self.vel_g = Rr2g @ self.Tw2t @ self.qd
        self.pos_g += self.vel_g * dt
        return self.pos_g

    def get_twist(self):
        return np.linalg.inv(self.Tt2w) @ self.qd

In [16]:
kin = Diff_Drive_Kinematic(r=0.075, b=0.4)

print(kin.get_wheelspeed([0.0, -1.0]))

for i in range(1000):
    print(kin.get_pose(1), kin.get_twist())

[ 2.66666667 -2.66666667]
[ 0.  0. -1.] [ 0. -1.]
[ 0.  0. -2.] [ 0. -1.]
[ 0.  0. -3.] [ 0. -1.]
[ 0.  0. -4.] [ 0. -1.]
[ 0.  0. -5.] [ 0. -1.]
[ 0.  0. -6.] [ 0. -1.]
[ 0.  0. -7.] [ 0. -1.]
[ 0.  0. -8.] [ 0. -1.]
[ 0.  0. -9.] [ 0. -1.]
[  0.   0. -10.] [ 0. -1.]
[  0.   0. -11.] [ 0. -1.]
[  0.   0. -12.] [ 0. -1.]
[  0.   0. -13.] [ 0. -1.]
[  0.   0. -14.] [ 0. -1.]
[  0.   0. -15.] [ 0. -1.]
[  0.   0. -16.] [ 0. -1.]
[  0.   0. -17.] [ 0. -1.]
[  0.   0. -18.] [ 0. -1.]
[  0.   0. -19.] [ 0. -1.]
[  0.   0. -20.] [ 0. -1.]
[  0.   0. -21.] [ 0. -1.]
[  0.   0. -22.] [ 0. -1.]
[  0.   0. -23.] [ 0. -1.]
[  0.   0. -24.] [ 0. -1.]
[  0.   0. -25.] [ 0. -1.]
[  0.   0. -26.] [ 0. -1.]
[  0.   0. -27.] [ 0. -1.]
[  0.   0. -28.] [ 0. -1.]
[  0.   0. -29.] [ 0. -1.]
[  0.   0. -30.] [ 0. -1.]
[  0.   0. -31.] [ 0. -1.]
[  0.   0. -32.] [ 0. -1.]
[  0.   0. -33.] [ 0. -1.]
[  0.   0. -34.] [ 0. -1.]
[  0.   0. -35.] [ 0. -1.]
[  0.   0. -36.] [ 0. -1.]
[  0.   0. -37.] [ 0. -1.]
[ 

In [18]:
a = np.ones(12)
a_len = len(a)
a_step = a_len // 3
a_ind = 3
for i in range(-1, 2):
    ind = (a_ind + (i * a_step)) % a_len
    print(ind)

11
3
7


In [ ]:
#!/usr/bin/python3

import rclpy
from rclpy.node import Node
from sensor_msgs.msg import LaserScan
from geometry_msgs.msg import Twist, Point, Vector3
from visualization_msgs.msg import Marker, MarkerArray
from std_msgs.msg import ColorRGBA
import numpy as np


class ModifyVFF_Avoidance(Node):
    def __init__(self):
        super().__init__('modify_vff_avoidance')
        self.create_subscription(LaserScan, "/scan", self.laser_scan_callback, 10)
        self.create_subscription(Vector3, "/attractive_vector", self.vector_callback, 10)
        self.create_timer(0.05, self.vff_controller)

        self.vel_pub = self.create_publisher(Twist, "/cmd_vel", 10)
        self.marker_pub = self.create_publisher(MarkerArray, "marker_debug", 10)

        self.laser_scan = None
        # Constants for visualization colors
        self.RED = 0
        self.GREEN = 1
        self.BLUE = 2

        self.OBSTACLE_DISTANCE = 2.0
        self.max_omega = 0.7
        self.max_vel = 0.5
        self.c = 1.0
        self.filt_const = 0.0

        self.vff_vector = {'attractive': [0.0, 0.0],  # Goal-directed vector
                      'repulsive': [0.0, 0.0],  # Obstacle avoidance vector
                      'result': [0.0, 0.0]}  # Combined vector

        # Callback for processing laser scan messages
    def laser_scan_callback(self, msg):
        # Store the laser scan data for use in the controller
        self.laser_scan = msg

    def vector_callback(self, msg : Vector3):
        norm = np.hypot(msg.x, msg.y)
        if norm <= 1:
            norm = 1
        self.vff_vector['attractive'][0] = msg.x / norm
        self.vff_vector['attractive'][1] = msg.y / norm

        # The main controller logic for VFF-based obstacle avoidance
    def vff_controller(self):
        if self.laser_scan != None:
            if np.hypot(self.vff_vector['attractive'][0], self.vff_vector['attractive'][1]) > 0.4:
                # print("Target: ", self.goal_x, self.goal_y)
                self.compute_vff(self.laser_scan)
                # Extract the resultant vector for calculating velocity commands
                angle = np.arctan2(self.vff_vector['result'][1], self.vff_vector['result'][0])
                module = np.hypot(self.vff_vector['result'][0], self.vff_vector['result'][1])
                # Create the velocity command message
                if np.fabs(angle) > np.radians(45):
                    module = 0.0
                else :
                    module = max(0, min(module, self.max_vel)) #Limit linear velocity
                angle = max(-self.max_omega, min(angle, self.max_omega)) #Limit angular velocity
                # Publish the velocity command
                self.publish_velocity(module, angle)

                # Publish visualization markers
                marker = self.get_debug_vff(self.vff_vector)
                self.marker_pub.publish(marker)
            else:
                self.publish_velocity(0.0, 0.0)

    
    # Calculate the Virtual Force Field based on laser scan data
    def compute_vff(self, scan : LaserScan):
        # Find the nearest obstacle
        min_idx = np.argmin(scan.ranges)
        distance_min = scan.ranges[min_idx]

        if distance_min < self.OBSTACLE_DISTANCE:
            angle = scan.angle_min + (scan.angle_increment * min_idx) 
            complementary_dist = self.OBSTACLE_DISTANCE - distance_min
            gain = self.c * complementary_dist / (self.OBSTACLE_DISTANCE)

        #     # Convert to Cartesian coordinates
            self.vff_vector['repulsive'][0] = ((1 - self.filt_const) * gain * complementary_dist * np.cos(angle + np.pi)) + (self.filt_const * self.vff_vector['repulsive'][0])
            self.vff_vector['repulsive'][1] = ((1 - self.filt_const) * gain * complementary_dist * np.sin(angle + np.pi)) + (self.filt_const * self.vff_vector['repulsive'][1])
        
        # Calculate the resultant vector by combining attractive and repulsive vectors
        self.vff_vector['result'][0] = self.vff_vector['attractive'][0] + self.vff_vector['repulsive'][0]
        self.vff_vector['result'][1] = self.vff_vector['attractive'][1] + self.vff_vector['repulsive'][1]

    def publish_velocity(self, linear_vel, angular_vel):
        twist = Twist()
        twist.linear.x = linear_vel
        twist.angular.z = angular_vel
        self.vel_pub.publish(twist)

    
    # Generate visualization markers for the VFF vectors
    def get_debug_vff(self, vff_vectors):
        marker_array = MarkerArray()
        # Create and add markers for attractive, repulsive, and resultant vectors
        marker_array.markers.append(self.make_marker(vff_vectors['attractive'], self.BLUE))
        marker_array.markers.append(self.make_marker(vff_vectors['repulsive'], self.RED))
        marker_array.markers.append(self.make_marker(vff_vectors['result'], self.GREEN))
        return marker_array

    # Utility function to create a marker for visualization
    def make_marker(self, vector, vff_color):
        marker = Marker()
        marker.header.frame_id = "base_link"
        marker.header.stamp = self.get_clock().now().to_msg()
        marker.type = Marker.ARROW
        marker.action = Marker.ADD
        # Define the start and end points of the marker
        start = Point(x=0.0, y=0.0, z=0.0)
        end = Point(x=vector[0], y=vector[1], z=0.0)
        marker.points = [start, end]
        # Set the scale of the marker
        marker.scale.x = 0.05
        marker.scale.y = 0.1
        marker.scale.z = 0.1
        # Set the color of the marker based on the type of vector
        color = ColorRGBA(a=1.0)  # Fully opaque
        if vff_color == self.RED:
            marker.id = 0
            color.r = 1.0
        elif vff_color == self.GREEN:
            marker.id = 1
            color.g = 1.0
        elif vff_color == self.BLUE:
            marker.id = 2
            color.b = 1.0
        marker.color = color
        return marker

def main(args=None):
    rclpy.init(args=args)
    node = ModifyVFF_Avoidance()
    rclpy.spin(node)
    node.destroy_node()
    rclpy.shutdown()

if __name__=='__main__':
    main()


In [ ]:
#!/usr/bin/python3

import rclpy
from rclpy.node import Node
from sensor_msgs.msg import LaserScan
from geometry_msgs.msg import Twist, Point, Vector3
from visualization_msgs.msg import Marker, MarkerArray
from std_msgs.msg import ColorRGBA
import numpy as np


class ModifyVFF_Avoidance(Node):
    def __init__(self):
        super().__init__('modify_vff_avoidance')
        self.create_subscription(LaserScan, "/scan", self.laser_scan_callback, 10)
        self.create_subscription(Vector3, "/attractive_vector", self.vector_callback, 10)
        self.create_timer(0.05, self.vff_controller)

        self.vel_pub = self.create_publisher(Twist, "/cmd_vel", 10)
        self.marker_pub = self.create_publisher(MarkerArray, "marker_debug", 10)

        self.laser_scan = None
        # Constants for visualization colors
        self.RED = 0
        self.GREEN = 1
        self.BLUE = 2

        self.OBSTACLE_DISTANCE = 2.0
        self.max_omega = 0.7
        self.max_vel = 0.5
        self.c = 1.0
        self.filt_const = 0.5
        self.div = 16

        self.vff_vector = {'attractive': [0.0, 0.0],  # Goal-directed vector
                      'repulsive': [0.0, 0.0],  # Obstacle avoidance vector
                      'result': [0.0, 0.0]}  # Combined vector

        # Callback for processing laser scan messages
    def laser_scan_callback(self, msg):
        # Store the laser scan data for use in the controller
        self.laser_scan = msg

    def vector_callback(self, msg : Vector3):
        norm = np.hypot(msg.x, msg.y)
        if norm <= 1:
            norm = 1
        self.vff_vector['attractive'][0] = msg.x
        self.vff_vector['attractive'][1] = msg.y

        # The main controller logic for VFF-based obstacle avoidance
    def vff_controller(self):
        if self.laser_scan != None:
            if np.hypot(self.vff_vector['attractive'][0], self.vff_vector['attractive'][1]) > 0.4:
                # print("Target: ", self.goal_x, self.goal_y)
                self.compute_vff(self.laser_scan)
                # Extract the resultant vector for calculating velocity commands
                angle = np.arctan2(self.vff_vector['result'][1], self.vff_vector['result'][0])
                module = np.hypot(self.vff_vector['result'][0], self.vff_vector['result'][1])
                # Create the velocity command message
                if np.fabs(angle) > np.radians(45):
                    module = 0.0
                else :
                    module = max(0, min(module, self.max_vel)) #Limit linear velocity
                angle = max(-self.max_omega, min(angle, self.max_omega)) #Limit angular velocity
                # Publish the velocity command
                self.publish_velocity(module, angle)

                # Publish visualization markers
                marker = self.get_debug_vff(self.vff_vector)
                self.marker_pub.publish(marker)
            else:
                self.publish_velocity(0.0, 0.0)

    
    # Calculate the Virtual Force Field based on laser scan data
    def compute_vff(self, scan : LaserScan):
        # Find the nearest obstacle
        min_idx = np.argmin(scan.ranges)
        idx_array = np.arange(0, len(scan.ranges), self.div)
        idx_array = np.append(idx_array, min_idx)
        repulsive_list = [0.0, 0.0]

        for i in idx_array:
            dis = scan.ranges[i]
            if dis < self.OBSTACLE_DISTANCE:
                ang = (i * scan.angle_increment) + scan.angle_min
                complementary_dist = self.OBSTACLE_DISTANCE - dis
                gain = self.c * complementary_dist / (self.OBSTACLE_DISTANCE ** 2)

                repulsive_list[0] += gain * complementary_dist * np.cos(ang + np.pi)
                repulsive_list[1] += gain * complementary_dist * np.sin(ang + np.pi)
        
        rep_ang = np.arctan2(repulsive_list[1], repulsive_list[0])
        if np.abs(rep_ang) > np.radians(60):
            hypot = np.hypot(repulsive_list[0], repulsive_list[1])
            hypot_ang = rep_ang * np.radians(60) / np.abs(rep_ang)
            repulsive_list[0] = hypot * np.cos(hypot_ang)
            repulsive_list[1] = hypot * np.sin(hypot_ang)

    #     # Convert to Cartesian coordinates
        self.vff_vector['repulsive'][0] = ((1 - self.filt_const) * repulsive_list[0]) + (self.filt_const * self.vff_vector['repulsive'][0])
        self.vff_vector['repulsive'][1] = ((1 - self.filt_const) * repulsive_list[1]) + (self.filt_const * self.vff_vector['repulsive'][1])
        
        # Calculate the resultant vector by combining attractive and repulsive vectors
        t = self.get_clock().now().to_msg().nanosec * 1.0e-9
        force_ripple = (0.3 * np.sin(2.0 * np.pi * 1.0 * t)) + 0.5
        self.vff_vector['result'][0] = self.vff_vector['attractive'][0] + force_ripple * self.vff_vector['repulsive'][0]
        self.vff_vector['result'][1] = self.vff_vector['attractive'][1] + force_ripple * self.vff_vector['repulsive'][1]

    def publish_velocity(self, linear_vel, angular_vel):
        twist = Twist()
        twist.linear.x = linear_vel
        twist.angular.z = angular_vel
        self.vel_pub.publish(twist)

    
    # Generate visualization markers for the VFF vectors
    def get_debug_vff(self, vff_vectors):
        marker_array = MarkerArray()
        # Create and add markers for attractive, repulsive, and resultant vectors
        marker_array.markers.append(self.make_marker(vff_vectors['attractive'], self.BLUE))
        marker_array.markers.append(self.make_marker(vff_vectors['repulsive'], self.RED))
        marker_array.markers.append(self.make_marker(vff_vectors['result'], self.GREEN))
        return marker_array

    # Utility function to create a marker for visualization
    def make_marker(self, vector, vff_color):
        marker = Marker()
        marker.header.frame_id = "base_link"
        marker.header.stamp = self.get_clock().now().to_msg()
        marker.type = Marker.ARROW
        marker.action = Marker.ADD
        # Define the start and end points of the marker
        start = Point(x=0.0, y=0.0, z=0.0)
        end = Point(x=vector[0], y=vector[1], z=0.0)
        marker.points = [start, end]
        # Set the scale of the marker
        marker.scale.x = 0.05
        marker.scale.y = 0.1
        marker.scale.z = 0.1
        # Set the color of the marker based on the type of vector
        color = ColorRGBA(a=1.0)  # Fully opaque
        if vff_color == self.RED:
            marker.id = 0
            color.r = 1.0
        elif vff_color == self.GREEN:
            marker.id = 1
            color.g = 1.0
        elif vff_color == self.BLUE:
            marker.id = 2
            color.b = 1.0
        marker.color = color
        return marker

def main(args=None):
    rclpy.init(args=args)
    node = ModifyVFF_Avoidance()
    rclpy.spin(node)
    node.destroy_node()
    rclpy.shutdown()

if __name__=='__main__':
    main()
